In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
import pickle
import youtokentome as yttm
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import ComplementNB
import numpy as np
from sklearn import metrics

In [3]:
with open('bow-language-detection.pkl', 'rb') as fopen:
    bow = pickle.load(fopen)

In [4]:
with open('train-test.json') as fopen:
    train_test = json.load(fopen)
    
train_test.keys()

dict_keys(['train_X', 'test_X', 'train_Y', 'test_Y'])

In [5]:
test_Y = LabelEncoder().fit_transform(train_test['test_Y'])

In [6]:
bpe = yttm.BPE(model='language-detection.model')

In [7]:
test_subs = [' '.join(s) for s in bpe.encode(train_test['test_X'], output_type=yttm.OutputType.SUBWORD)]
len(test_subs)

2248403

In [8]:
test_X = bow.transform(test_subs)

In [9]:
import tensorflow as tf

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [10]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensorValue(indices, coo.col, coo.shape), tf.SparseTensorValue(indices, coo.data, coo.shape)

In [13]:
class Model:
    def __init__(self, learning_rate, dimension = 64, output = 6):
        self.X = tf.sparse_placeholder(tf.int32)
        self.W = tf.sparse_placeholder(tf.int32)
        self.Y = tf.placeholder(tf.int32, [None])
        embeddings = tf.Variable(tf.truncated_normal([300000,dimension]))
        embed = tf.nn.embedding_lookup_sparse(embeddings, self.X, self.W, combiner='mean')
        self.embed = embed
        self.logits = tf.layers.dense(embed, output)
        self.cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1,output_type=tf.int32), self.Y)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [14]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(1e-3)
sess.run(tf.global_variables_initializer())

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [15]:
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, 'lang-detection-w/model.ckpt')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from lang-detection-w/model.ckpt


In [17]:
from tqdm import tqdm

batch_size = 128
predicted_Y = []

for i in tqdm(range(0, test_X.shape[0], batch_size)):
    index = min(i + batch_size, test_X.shape[0])
    batch_x = convert_sparse_matrix_to_sparse_tensor(test_X[i: index])
    results = sess.run(model.logits, feed_dict = {
                model.X: batch_x[0],
                model.W: batch_x[1],
            })
    predicted_Y.extend(np.argmax(results, axis = 1))

100%|██████████| 17566/17566 [00:32<00:00, 544.81it/s]


In [18]:
len(test_Y) == len(predicted_Y)

True

In [22]:
print(
    metrics.classification_report(
        test_Y,
        predicted_Y,
        target_names = ['eng', 'ind', 'malay', 'manglish', 'other', 'rojak'],
        digits = 5
    )
)

              precision    recall  f1-score   support

         eng    0.97682   0.97848   0.97765    110313
         ind    0.93604   0.92041   0.92816    622928
       malay    0.94543   0.96712   0.95615   1045233
    manglish    0.98813   0.98861   0.98837    169901
       other    0.98187   0.97204   0.97693     66198
       rojak    0.96127   0.90709   0.93339    233830

    accuracy                        0.95026   2248403
   macro avg    0.96493   0.95562   0.96011   2248403
weighted avg    0.95032   0.95026   0.95013   2248403

